In [1]:
import torch
from reconstruction import AE
from datasets import MeshData
from utils import utils, DataLoader, mesh_sampling, sap
import numpy as np
import pyvista as pv
from skimage import measure
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
from IPython.display import display
import meshplot as mp
import os, sys
from math import ceil
from scipy.ndimage import zoom
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Meshplot left an annoying print statement in their code. Using this context manager to supress it...
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [3]:
device = torch.device('cuda', 0)
# Set the path to the saved model directory
model_path = "/home/jakaria/Explaining_Shape_Variability/src/DeepLearning/compute_canada/guided_vae/data/CoMA/raw/torus/models/1"

# Load the saved model
model_state_dict = torch.load(f"{model_path}/model_state_dict.pt")
in_channels = torch.load(f"{model_path}/in_channels.pt")
out_channels = torch.load(f"{model_path}/out_channels.pt")
latent_channels = torch.load(f"{model_path}/latent_channels.pt")
spiral_indices_list = torch.load(f"{model_path}/spiral_indices_list.pt")
up_transform_list = torch.load(f"{model_path}/up_transform_list.pt")
down_transform_list = torch.load(f"{model_path}/down_transform_list.pt")
std = torch.load(f"{model_path}/std.pt")
mean = torch.load(f"{model_path}/mean.pt")
template_face = torch.load(f"{model_path}/faces.pt")

# Create an instance of the model
model = AE(in_channels, out_channels, latent_channels,
           spiral_indices_list, down_transform_list,
           up_transform_list)
model.load_state_dict(model_state_dict)
model.to(device)
# Set the model to evaluation mode
model.eval()

AE(
  (en_layers): ModuleList(
    (0): SpiralEnblock(
      (conv): SpiralConv(3, 16, seq_length=9)
    )
    (1-2): 2 x SpiralEnblock(
      (conv): SpiralConv(16, 16, seq_length=9)
    )
    (3): SpiralEnblock(
      (conv): SpiralConv(16, 32, seq_length=9)
    )
    (4): Linear(in_features=3136, out_features=16, bias=True)
  )
  (de_layers): ModuleList(
    (0): Linear(in_features=8, out_features=3136, bias=True)
    (1): SpiralDeblock(
      (conv): SpiralConv(32, 32, seq_length=9)
    )
    (2): SpiralDeblock(
      (conv): SpiralConv(32, 16, seq_length=9)
    )
    (3-4): 2 x SpiralDeblock(
      (conv): SpiralConv(16, 16, seq_length=9)
    )
    (5): SpiralConv(16, 3, seq_length=9)
  )
  (cls_sq): Sequential(
    (0): Linear(in_features=1, out_features=8, bias=True)
    (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
    (3): Linear(in_features=8, out_features=8, bias=True)
    (4): BatchNo

In [7]:
z = torch.zeros(16)
plot=None
@mp.interact(**{f'z[{i}]': FloatSlider(min=-2.0, max=2.0, step=0.2, value=0) for i in range(16)})
def show(**kwargs):
    global plot
    global z
    z = torch.tensor([kwargs[f'z[{i}]'] for i in range(16)])
    with torch.no_grad():
        z = z.to(device)
        #print(z)
        pred = model.decoder(z)

        reshaped_pred = (pred.view(-1, 3).cpu() * std) + mean
        reshaped_pred = reshaped_pred.cpu().numpy()
        print(reshaped_pred.shape)

    verts = reshaped_pred
    pcd = o3d.io.read_triangle_mesh('/home/jakaria/Explaining_Shape_Variability/src/DeepLearning/compute_canada/guided_vae/data/CoMA/template/template.ply')
    faces = np.asarray(pcd.triangles)

    if plot is None:
        plot = mp.plot(verts, faces, return_plot=True)
    else:
        with HiddenPrints():
            plot.update_object(vertices=verts, faces=faces)
        display(plot._renderer)

interactive(children=(FloatSlider(value=0.0, description='z[0]', max=2.0, min=-2.0, step=0.2), FloatSlider(val…

In [23]:
latent_channels = torch.load(f"{model_path}/latent_channels.pt")
angles = torch.load(f"{model_path}/angles.pt")

In [4]:
template_fp = "/home/jakaria/Explaining_Shape_Variability/src/DeepLearning/compute_canada/guided_vae/data/CoMA/template/template.ply"
data_fp = "/home/jakaria/Explaining_Shape_Variability/src/DeepLearning/compute_canada/guided_vae/data/CoMA"
test_exp = "bareteeth"
split = "interpolation"

meshdata = MeshData(data_fp,
                    template_fp,
                    split=split,
                    test_exp=test_exp)

test_loader = DataLoader(meshdata.test_dataset, batch_size=1)

reshaped_pred_all = []
reshaped_x_all = []

with torch.no_grad():
    for i, data in enumerate(test_loader):
        #print("test...")
        x = data.x.to(device)
        y = data.y.to(device)
        pred, mu, log_var, re = model(x)
    
        num_graphs = data.num_graphs
        reshaped_pred = (pred.view(num_graphs, -1, 3).cpu() * std) + mean
        reshaped_x = (x.view(num_graphs, -1, 3).cpu() * std) + mean
        
        reshaped_pred = reshaped_pred.cpu().numpy()
        reshaped_x = reshaped_x.cpu().numpy()

        reshaped_pred_all.append(reshaped_pred)
        reshaped_x_all.append(reshaped_x)

/home/jakaria/anaconda3/envs/pytorch_geo/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The data of the dataset is already cached, so any modifications to `data` will not be reflected when accessing its elements. Clearing the cache now by removing all elements in `dataset._data_list`. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/home/jakaria/anaconda3/envs/pytorch_geo/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage

Normalizing...
Done!


In [18]:
verts = reshaped_pred_all[2]
verts_x = reshaped_x_all[2]
pcd = o3d.io.read_triangle_mesh('/home/jakaria/Explaining_Shape_Variability/src/DeepLearning/compute_canada/guided_vae/data/CoMA/template/template.ply')
faces = np.asarray(pcd.triangles)

In [24]:
white_color = [1.0, 1.0, 1.0]
grey_color = [0.5, 0.5, 0.5]
if plot is None:
    plot = mp.plot(verts, faces, c=np.array(grey_color), return_plot=True)
else:
    with HiddenPrints():
        plot.update_object(vertices=verts, faces=faces)
    display(plot._renderer)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(49.376184…